# Step 8: Model Deployment Using FastAPI

**Process and Purpose**  
1. We specify the exact run ID and model name from MLflow to load the best performing model.  
2. We define a FastAPI application for serving predictions.  
3. The “app.py” file includes:  
   - A request schema class “ChurnRequest” to parse input data.  
   - An endpoint “/predict” that transforms the request into a DataFrame, then predicts with the loaded model.  
   - A basic “/” route returning a “Hello World” message.  

**Key Steps**  
- Place the correct MLflow run URI combining the run ID and model name.  
- Load the PyFunc model with “mlflow.pyfunc.load_model.”  
- Implement a FastAPI server that listens for JSON requests, converts them to a DataFrame, and returns a churn prediction.  
- Launch the server via “uvicorn app:app --host 0.0.0.0 --port 8000.”  

In [14]:
# %pip --quiet install fastapi uvicorn mlflow pydantic

In [1]:
import mlflow.pyfunc

# Use the correct Run ID
run_id = "d0f7d16ce56d49d3868601fc9180321d"
model_name = "RandomForest"  # Change if another model performed better

# Correct MLflow URI
mlflow_uri = f"runs:/{run_id}/{model_name}"

# Load the model
model = mlflow.pyfunc.load_model(mlflow_uri)

print(f"✅ Model '{model_name}' loaded successfully from MLflow (Run ID: {run_id})")


✅ Model 'RandomForest' loaded successfully from MLflow (Run ID: d0f7d16ce56d49d3868601fc9180321d)


In [ ]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
import mlflow.pyfunc
import pandas as pd

# Initialize FastAPI app
app = FastAPI()

# Load best model from MLflow
mlflow_uri = "runs:/d0f7d16ce56d49d3868601fc9180321d/RandomForest"
model = mlflow.pyfunc.load_model(mlflow_uri)


# Define request schema
class ChurnRequest(BaseModel):
    Call_Failure: int
    Complains: int
    Subscription_Length: int
    Charge_Amount: int
    Seconds_of_Use: int
    Frequency_of_use: int
    Frequency_of_SMS: int
    Distinct_Called_Numbers: int
    Age_Group: int
    Tariff_Plan: int
    Status: int
    Age: int
    Customer_Value: float


# Define API endpoint
@app.get("/")
def root():
    """
    Root endpoint that returns a simple greeting message.
    This serves as a health check endpoint to verify the API is running.

    Returns:
        dict: A simple JSON message indicating the API is working
    """
    return {"message": "Hello World"}


@app.post("/predict")
def predict_churn(request: ChurnRequest):
    """
    Predict customer churn based on input features.

    This endpoint receives customer data in the form of a ChurnRequest,
    processes it into a format suitable for the model, and returns
    a binary prediction indicating whether the customer is likely to churn.

    Args:
            request (ChurnRequest): The customer data containing all required features
                                                       for making a churn prediction

    Returns:
            dict: A dictionary containing the churn prediction (0 = no churn, 1 = churn)
    """
    # Convert request data to DataFrame
    print("=====REQUEST======", request)
    data = pd.DataFrame([request.model_dump()])

    # Make prediction
    prediction = model.predict(data)

    # Return result
    return {"churn_prediction": int(prediction[0])}


# To run, execute this in terminal:
# uvicorn app:app --host 0.0.0.0 --port 8000


Overwriting app.py
